In [1]:
%pip install pandas yfinance tqdm eikon

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import yfinance as yf
from tqdm import tqdm
import numpy as np
import eikon as ek
import pandas as pd

# Set your Eikon API key
ek.set_app_key('e1df9a44dc6c4206a0599ca61d351c5a92689f71')

/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
input_file = '/Users/christopher/Desktop/Seminar-Advanced-Finance/stocks.csv'

# Read the CSV file
stocks_df = pd.read_csv(input_file)

In [5]:
# We exclude the 'Financials' fr
List = ['Health Care',
'Information Technology',
'Consumer Discretionary',
'Consumer Staples',
'Industrials',
'Utilities',
'Materials',
'Real Estate',
'Energy',
'Communication Services']

stocks_df = stocks_df[stocks_df['GICS Sector'].isin(List)]

# List to accumulate data frames for each ticker
final_df2 = pd.DataFrame()

In [6]:
# Create a list of tickers with the "@ticker" suffix
RICS_list = [symbol + "@ticker" for symbol in stocks_df['Symbol'].tolist()]

# Retrieve tickers information using Eikon API
tickers, err = ek.get_data(RICS_list, 'TR.RIC')

/Users/christopher/opt/anaconda3/lib/python3.9/site-packages/eikon/data_grid.py:255: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


In [8]:
tickers.rename(columns={'Instrument': 'Symbol'}, inplace=True)
tickers = pd.merge(tickers, stocks_df, on='Symbol', how='left')
tickers.rename(columns={'Symbol': 'Instrument'}, inplace=True)

In [9]:
USGDP = ek.get_timeseries('aUSGDPCC/CA', 
                       start_date="1999-01-01", 
                       end_date="2025-12-31", 
                       interval="quarterly")
USGDP['Return'] = USGDP['VALUE'] / USGDP['VALUE'].shift(1) - 1

In [10]:
# Define the fields to retrieve for quarterly fundamentals
fields = [
    'TR.TotalEquity(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ).periodenddate',
    'TR.TotalEquity(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.NetIncome(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.TtlLiabShareholderEqty(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.IncomeTaxRatePct(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.TotalReceivablesNet(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.INVNTTOT(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.PPENETTOT(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.Revenue(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.DIVYLDCOMSTOCKISSUEPCTTTM(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.NETINCAFTERMININTR(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.MKTCAP(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.MKTCAP3YRAVG(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.TOTDEBTPCTOFTOTASSETS(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.TOTDEBTPCTOFTOTEQ(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.TOTDEBTPCTOFTOTCAP(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.PPEEXCLROUTANGCAPLEASENETSUPPL(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.RND(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.PRICECLOSEENDPRD(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.PRICETOEPSDILEXCLEXORDNORMTOTTTM(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.EBITDAMARGPCT(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.F.DEBTTOT(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)',
    'TR.TotalAssets(SDate="1999-01-01",EDate="2025-08-01",Period=FQ0,Frq=FQ)'
]

# Define parameters: Scale adjusts the numeric magnitude and Curn sets the currency.
params = {'Scale': 6, 'Curn': 'USD'}

# Initialize an empty DataFrame to store all tickers' fundamental data
stocksinfo_df = pd.DataFrame()

# Iterate over each ticker in the tickers DataFrame using the 'RIC' column
for idx, row in tqdm(tickers.iterrows(), total=tickers.shape[0], desc="Downloading fundamentals"):
    ticker = row['RIC']  # Assuming your tickers DataFrame has a column named 'RIC'
    GICS = row['GICS Sector']
    data, err = ek.get_data(ticker, fields, parameters=params)
    if err:
        print(f"Error retrieving data for {ticker}: {err}")
        continue
    temp_df = pd.DataFrame(data)
    # Add the ticker identifier to the temporary DataFrame
    temp_df['Ticker'] = ticker
    temp_df['GICS Sector'] = GICS
    stocksinfo_df = pd.concat([stocksinfo_df, temp_df], ignore_index=True)

  df = df.apply(pd.to_numeric, errors='ignore')

Error retrieving data for AMRG.PK^H09: [{'code': 412, 'col': 1, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 2, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 3, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 4, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 5, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 6, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 7, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 8, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 9, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 10, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col'

  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='i

Error retrieving data for CABE.PK^L24: [{'code': 416, 'col': 7, 'message': "Unable to collect data for the field 'TR.F.InvntTot' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 8, 'message': "Unable to collect data for the field 'TR.F.PPENetTot' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 10, 'message': "Unable to collect data for the field 'TR.F.DivYldComStockIssuePctTTM' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 11, 'message': "Unable to collect data for the field 'TR.F.NetIncAfterMinIntr' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 12, 'message': "Unable to collect data for the field 'TR.F.MktCap' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 13, 'message': "Unable to collect data for the field 'TR.F.MktCap3YrAvg' and some specific identifier(s).", 'row': 0}, {'code': 416, 'col': 14, 'message': "Unable to collect data for the field 'TR.F.TotDebtPctofTotAssets' and some spec

  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='i

Error retrieving data for ES^J99: [{'code': 412, 'col': 1, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 2, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 3, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 4, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 5, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 6, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 7, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 8, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 9, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 10, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 11,

  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')

Error retrieving data for FEEN.PK^H10: [{'code': 412, 'col': 1, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 2, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 3, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 4, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 5, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 6, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 7, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 8, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 9, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col': 10, 'message': 'Unable to resolve all requested identifiers.', 'row': 0}, {'code': 412, 'col'

  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='ignore')
  df = df.apply(pd.to_numeric, errors='i

In [11]:
stocksinfo_df.to_csv('/Users/christopher/Desktop/Seminar-Advanced-Finance/stocksinfo_data.csv', index=False)

In [12]:
# Convert 'Period End Date' in stocksinfo_df to datetime if it isn’t already
stocksinfo_df['Period End Date'] = pd.to_datetime(stocksinfo_df['Period End Date'])
# Create a new column representing the quarter period
stocksinfo_df['Period'] = stocksinfo_df['Period End Date'].dt.to_period('Q')

# Ensure USGDP's date column is datetime. If USGDP’s date is the index, you can reset the index:
if 'Date' not in USGDP.columns:
    USGDP = USGDP.reset_index()
USGDP['Date'] = pd.to_datetime(USGDP['Date'])
# Create a quarterly period column for USGDP as well
USGDP['Period'] = USGDP['Date'].dt.to_period('Q')

# Now merge on the 'Period' column
merged_df = pd.merge(stocksinfo_df, USGDP, on='Period', how='left')

stocksinfo_df = merged_df

In [13]:
# Fjern rækker hvor 'AdjustedDate' er NaT eller har årstal under 2000
stocksinfo_df = stocksinfo_df[stocksinfo_df['Period End Date'].notnull() & (stocksinfo_df['Period End Date'].dt.year >= 2000)]

In [14]:
stocksinfo_df["Debt Ratio"] = (
    (stocksinfo_df["Total Liabilities And Shareholders' Equity"] - stocksinfo_df["Total Equity"]) /
    stocksinfo_df["Total Assets"]
)

# Compute ROE:
# ROE = Net Income Incl Extra Before Distributions / Total Equity
stocksinfo_df["ROE"] = (
    stocksinfo_df["Net Income Incl Extra Before Distributions"] / stocksinfo_df["Total Equity"]
)

# Compute Asset Tangibility:
# Asset Tangibility = (Property Plant & Equipment - Net - Total + Inventories - Total) / Total Assets
stocksinfo_df["Property Plant & Equipment - Net - Total"] = stocksinfo_df["Property Plant & Equipment - Net - Total"].fillna(0)
stocksinfo_df["Asset Tangibility"] = (
    (stocksinfo_df["Property Plant & Equipment - Net - Total"] + stocksinfo_df["Inventories - Total"]) /
    stocksinfo_df["Total Assets"]
)

stocksinfo_df["TobinsQ"] = (
    (stocksinfo_df["Market Capitalization"]) /
    stocksinfo_df["Total Assets"]
)

stocksinfo_df["RnDintensity"] = (
    (stocksinfo_df["Research & Development Expense"]) /
    stocksinfo_df["Total Assets"]
)

stocksinfo_df["PEratio"] = (
    (stocksinfo_df["Price Close (End of Period)"]) /
    stocksinfo_df["Price to EPS Diluted excl Exord - Norm - Total, TTM"]
)

stocksinfo_df['Size'] = np.log(stocksinfo_df['Total Assets'])
stocksinfo_df.rename(columns={'Dividend Yield - Common - Net - Issue - %, TTM': 'DPR'}, inplace=True)
stocksinfo_df.rename(columns={'Net Income after Minority Interest': 'Net Income'}, inplace=True)

stocksinfo_df.rename(columns={'VALUE': 'GDP'}, inplace=True)
stocksinfo_df.rename(columns={'Return': 'GDP Growth'}, inplace=True)
stocksinfo_df.rename(columns={'EBITDA Margin - %': 'EBITDAmargin'}, inplace=True)

/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_94536/1314832348.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksinfo_df["Debt Ratio"] = (
/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_94536/1314832348.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksinfo_df["ROE"] = (
/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_94536/1314832348.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [15]:
import pandas as pd
import yfinance as yf

# Download daily VIX data
vix_data = yf.download("^VIX", start="1999-01-01", end="2025-12-31")

# Resample to quarterly using the last closing value of each quarter
vix_quarterly = vix_data[['Close']].resample('Q').last()

# Reset index to ensure it's a single-level DataFrame
vix_quarterly = vix_quarterly.reset_index()

# Convert 'Date' column to quarterly periods
vix_quarterly['Period'] = vix_quarterly['Date'].dt.to_period('Q')

# 🔹 FIX: Flatten MultiIndex if necessary
if isinstance(vix_quarterly.columns, pd.MultiIndex):
    vix_quarterly.columns = [col[0] for col in vix_quarterly.columns]

# Keep only 'Period' and 'Close', and rename 'Close' to 'VIX_Close'
vix_quarterly = vix_quarterly[['Period', 'Close']].rename(columns={'Close': 'VIX_Close'})

# 🔹 Beregn VIX Growth Rate (kvartal-til-kvartal ændring)
vix_quarterly['VIX_Growth'] = vix_quarterly['VIX_Close'].pct_change()

# Convert 'Period End Date' in stocksinfo_df to datetime and extract quarterly period
stocksinfo_df['Period'] = pd.to_datetime(stocksinfo_df['Period End Date']).dt.to_period('Q')

# Convert Period to string in both DataFrames to ensure matching format
stocksinfo_df['Period'] = stocksinfo_df['Period'].astype(str)
vix_quarterly['Period'] = vix_quarterly['Period'].astype(str)

# Merge stocksinfo_df with vix_quarterly on 'Period'
stocksinfo_df = pd.merge(stocksinfo_df, vix_quarterly, on='Period', how='left')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_94536/646556795.py:8: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  vix_quarterly = vix_data[['Close']].resample('Q').last()
/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_94536/646556795.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksinfo_df['Period'] = pd.to_datetime(stocksinfo_df['Period End Date']).dt.to_period('Q')
/var/folders/1t/g6pbwy2d0xs212cq6dztdzn80000gn/T/ipykernel_94536/646556795.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [16]:
import pandas as pd
import pandas_datareader.data as web
import datetime

# Define date range
start = datetime.datetime(1999, 1, 1)
end = datetime.datetime(2025, 12, 31)

# Get U.S. Fed Funds Rate from FRED
interest_rate_data = web.DataReader("FEDFUNDS", "fred", start, end)

# Convert index to a column
interest_rate_data.reset_index(inplace=True)

# Convert 'DATE' column to quarterly period
interest_rate_data['Period'] = interest_rate_data['DATE'].dt.to_period('Q')

# Rename columns
interest_rate_data.rename(columns={"FEDFUNDS": "Interest Rate"}, inplace=True)

# Sort by DATE in descending order to ensure the latest is first
interest_rate_data = interest_rate_data.sort_values(by="DATE", ascending=False)

# Keep only the latest entry for each Period
interest_rate_quarterly = interest_rate_data.drop_duplicates(subset=["Period"], keep="first")

# Reset index after filtering
interest_rate_quarterly = interest_rate_quarterly.reset_index(drop=True)

# Ensure both Period columns are in the same format (string)
stocksinfo_df['Period'] = stocksinfo_df['Period'].astype(str)
interest_rate_quarterly['Period'] = interest_rate_quarterly['Period'].astype(str)

# Merge stocksinfo_df with interest_rate_quarterly on 'Period'
stocksinfo_df = pd.merge(stocksinfo_df, interest_rate_quarterly, on='Period', how='left')

In [17]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

In [18]:
# Replace any +inf or -inf with NaN in your entire panel_data
final_merged_df = stocksinfo_df.replace([np.inf, -np.inf], np.nan)

In [19]:
final_merged_df['DebtRatio_Lag'] = final_merged_df.groupby('Instrument')['Debt Ratio'].shift(1)
final_merged_df['VIX_Growth_Lag'] = final_merged_df.groupby('Instrument')['VIX_Growth'].shift(1)
final_merged_df['VIX_Growth_Lag2'] = final_merged_df.groupby('Instrument')['VIX_Growth'].shift(2)
final_merged_df['Debt_VIX_Interaction'] = final_merged_df['Debt Ratio'] * final_merged_df['VIX_Growth']
final_merged_df['Log_TobinsQ'] = np.log1p(final_merged_df['TobinsQ'])
final_merged_df['Debt_Size_Interaction'] = final_merged_df['Debt Ratio'] * final_merged_df['Size']
final_merged_df['Log_DebtRatio'] = np.log1p(final_merged_df['Debt Ratio'])  # log(1 + Debt Ratio)
final_merged_df['Log_Debt_Size_Interaction'] = final_merged_df['Log_DebtRatio'] * final_merged_df['Size']
final_merged_df['DebtRatio_sq'] = final_merged_df['Debt Ratio'] ** 2
final_merged_df["RnDintensity"] = final_merged_df["RnDintensity"].fillna(0)
final_merged_df = final_merged_df[final_merged_df['Instrument'] != 'SIOX.PK^B24']

In [20]:
panel_data = final_merged_df.set_index(['Instrument', 'Period End Date'])
xx = panel_data[['Log_TobinsQ','Debt Ratio' , 'ROE', 'Asset Tangibility', 'Size', 'DPR', 'Net Income', 'GDP Growth', 'VIX_Growth', 'RnDintensity', 'PEratio', 'Interest Rate', 'EBITDAmargin']]

In [21]:
import pandas as pd

# Assuming xx is your dataframe
summary_stats = xx.describe().T  # Transpose to match LaTeX table format

# Round numbers for better readability
summary_stats = summary_stats[['mean', 'std', 'min', 'max']].round(3)

# Rename standard deviation column
summary_stats.rename(columns={'std': 'sd'}, inplace=True)

# Add the observation row properly
summary_stats.loc["Observations"] = [f"{xx.shape[0]:,}", "", "", ""]

# Convert to LaTeX with proper formatting
latex_code = summary_stats.to_latex(
    column_format="lcccc",  # Ensures alignment
    escape=False,
    caption="Deskriptiv statistik",
    label="tab:summary"
)

# Ensure Observations row is well-placed
latex_code = latex_code.replace(
    r"\bottomrule",
    f"Observations & {xx.shape[0]:,} & & & \\\\\n\ bottomrule"
)

# Save to file
with open("summary_statistics.tex", "w") as f:
    f.write(latex_code)

print(latex_code)  # Print LaTeX table


\begin{table}
\caption{Deskriptiv statistik}
\label{tab:summary}
\begin{tabular}{lcccc}
\toprule
 & mean & sd & min & max \\
\midrule
Log_TobinsQ & 0.934864 & 0.582587 & 0.000000 & 6.264759 \\
Debt Ratio & 0.612235 & 0.269605 & 0.023229 & 4.470956 \\
ROE & 0.032141 & 1.537198 & -104.611111 & 140.339244 \\
Asset Tangibility & 0.407489 & 0.290454 & -24.315227 & 2.899001 \\
Size & 9.184701 & 1.638027 & -1.170391 & 13.649442 \\
DPR & 1.442720 & 2.095226 & -67.398119 & 52.528093 \\
Net Income & 461.301943 & 1681.963008 & -31115.000000 & 127140.000000 \\
GDP Growth & 0.003516 & 0.012947 & -0.079708 & 0.077751 \\
VIX_Growth & 0.055961 & 0.441164 & -0.460661 & 2.885341 \\
RnDintensity & 0.006692 & 0.014903 & -0.106468 & 0.394303 \\
PEratio & 4.760252 & 107.097480 & 0.000370 & 18978.128940 \\
Interest Rate & 1.899301 & 2.009538 & 0.070000 & 6.530000 \\
EBITDAmargin & -48.435450 & 11975.405910 & -2231000.000000 & 3412.289160 \\
Observations & 34,999 &  &  &  \\
Observations & 34,999 & & & \\
\ b

In [22]:
final_merged_df.to_csv('/Users/christopher/Desktop/Seminar-Advanced-Finance/finaldata.csv', index=False)